In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [4]:
from companies.models import Company

In [5]:
Company.objects.create(name="Company AAA", num_employees=120, num_chairs=150, num_tables=60)
Company.objects.create(name="Company BBB", num_employees=50, num_chairs=30, num_tables=20)
Company.objects.create(name="Company CCC", num_employees=100, num_chairs=40, num_tables=40)

<Company: Company CCC>

In [7]:
from django.db.models import Count, F, Value
from django.db.models.functions import Length, Upper
from django.db.models.lookups import GreaterThan

In [10]:
# Find companies that have more employees than chairs.
qs = Company.objects.filter(num_employees__gt=F("num_chairs"))

In [11]:
print(qs.query)

SELECT "companies_company"."id", "companies_company"."name", "companies_company"."ticker", "companies_company"."num_employees", "companies_company"."num_tables", "companies_company"."num_chairs" FROM "companies_company" WHERE "companies_company"."num_employees" > ("companies_company"."num_chairs")


In [12]:
# Find companies that have at least twice as many employees as chairs.
qs = Company.objects.filter(num_employees__gt=F("num_chairs") * 2)

In [13]:
print(qs.query)

SELECT "companies_company"."id", "companies_company"."name", "companies_company"."ticker", "companies_company"."num_employees", "companies_company"."num_tables", "companies_company"."num_chairs" FROM "companies_company" WHERE "companies_company"."num_employees" > ("companies_company"."num_chairs" * 2)


## Annotation

In [19]:
# How many chairs are needed for each company to seat all employees?
company = (
    Company.objects.filter(num_employees__gt=F("num_chairs"))
    .annotate(chairs_needed=F("num_employees") - F("num_chairs"))
)

In [20]:
for c in company:
    print(f"{c.name}: Chair Needed = {c.chairs_needed}")

Company BBB: Chair Needed = 20
Company CCC: Chair Needed = 60


In [26]:
company = Company.objects.annotate(chairs_needed=F("num_employees") - F("num_chairs")).filter(chairs_needed__gt=50)


In [27]:
for c in company:
    print(f"{c.name}: Chair Needed = {c.chairs_needed}")

Company CCC: Chair Needed = 60


In [28]:
print(company.query)

SELECT "companies_company"."id", "companies_company"."name", "companies_company"."ticker", "companies_company"."num_employees", "companies_company"."num_tables", "companies_company"."num_chairs", ("companies_company"."num_employees" - "companies_company"."num_chairs") AS "chairs_needed" FROM "companies_company" WHERE ("companies_company"."num_employees" - "companies_company"."num_chairs") > 50


In [35]:
company = Company.objects.create(name="Google", ticker=Upper(Value("goog")), num_employees=10, num_tables=20, num_chairs=30)

In [38]:
# Order By
Company.objects.all().order_by("-name", "num_employees")

<QuerySet [<Company: Google>, <Company: Company CCC>, <Company: Company BBB>, <Company: Company AAA>]>

In [39]:
Company.objects.all().order_by(Length("name").asc())

<QuerySet [<Company: Google>, <Company: Company AAA>, <Company: Company BBB>, <Company: Company CCC>]>

In [40]:
Company.objects.all().order_by(Length("name").desc())

<QuerySet [<Company: Company AAA>, <Company: Company BBB>, <Company: Company CCC>, <Company: Google>]>

In [41]:
# Lookup expressions can also be used directly in filters
Company.objects.filter(GreaterThan(F("num_employees"), F("num_chairs")))
# or annotations.
Company.objects.annotate(
    need_chairs=GreaterThan(F("num_employees"), F("num_chairs")),
)

<QuerySet [<Company: Company AAA>, <Company: Company BBB>, <Company: Company CCC>, <Company: Google>]>

# Books

In [2]:
from books.models import Book

In [5]:
# Total number of books.
Book.objects.count()

59

In [6]:
# Total number of books with publisher=Penguin Books
Book.objects.filter(publisher__name="Penguin Books").count()

20

In [7]:
Book.objects.aggregate(Avg("price", default=0))

{'price__avg': Decimal('9.7018644067796610')}

In [8]:
Book.objects.aggregate(Max("price", default=0))

{'price__max': Decimal('14.99')}

## Many to Many

In [7]:
# Get all books published by "Penguin Books"
penguin_pub = Publisher.objects.get(name="Penguin Books")
penguin_pub.book_set.all().count()

20

In [9]:
# Get books with name starting with "The"
qs = penguin_pub.book_set.filter(name__startswith="The")
print(qs.query)

SELECT "books_book"."id", "books_book"."name", "books_book"."pages", "books_book"."price", "books_book"."rating", "books_book"."publisher_id", "books_book"."pubdate" FROM "books_book" WHERE ("books_book"."publisher_id" = 1 AND "books_book"."name"::text LIKE The%)


In [12]:
# Get only ids
qs = penguin_pub.book_set.filter(name__startswith="The").values_list("id", flat=True)

In [13]:
list(qs)

[1, 5, 8, 14, 17]

In [11]:
# Get id and name
penguin_pub.book_set.filter(name__startswith="The").values("id", "name")

<QuerySet [{'id': 1, 'name': 'The Great Gatsby'}, {'id': 5, 'name': 'The Catcher in the Rye'}, {'id': 8, 'name': 'The Odyssey'}, {'id': 14, 'name': 'The Hobbit'}, {'id': 17, 'name': 'The Hitchhiker Guide to the Galaxy'}]>